# OpenAI driven search of ICWA Legistation
This uses the Western Australian [Strata Titles Act 1985](https://www.legislation.wa.gov.au/legislation/statutes.nsf/main_mrtitle_938_homepage.html)


# Initialisation

Now load the .env file to get the API keys in a secure way. The path should be the full path to the .env file. If this work it returns `True`

In [1]:
from dotenv import load_dotenv
load_dotenv() #get API keys

True

# Prepare legislation

The legislation is a word document that can be readily manipulated using the `docx` module.

In [2]:
import docx
import re
legislation_path = r'documents/Strata Titles Act 1985.docx'

Based on the following we see the document uses 4 levels of headings as follows
- Heading 2: The Parts of the legislation.
- Heading 3: Divisions
- Heading 5: These are used as to create sub-headings in the level 2 & 3 headings. 

The cover page and TOC are the first part of document and are discarded. 

In [3]:
headings = tuple(set( paragraph.style.name for paragraph in docx.Document(legislation_path).paragraphs if paragraph.style.name.startswith("Heading")))
print(sorted(headings))
toc = [": ".join([paragraph.style.name, re.sub(r"\s+", ' ', paragraph.text)]) for paragraph in docx.Document(legislation_path).paragraphs if paragraph.style.name.startswith("Heading")]
toc

['Heading 2', 'Heading 3', 'Heading 4', 'Heading 5']


['Heading 2: Part 1 — Preliminary',
 'Heading 5: 1. Short title',
 'Heading 5: 2. Commencement',
 'Heading 5: 3. Terms used',
 'Heading 5: 4. Notes and examples not part of Act',
 'Heading 5: 5. Act binds Crown',
 'Heading 2: Part 2 — Strata titles schemes',
 'Heading 5: 6. Legislative framework',
 'Heading 5: 7. Strata titles schemes',
 'Heading 5: 8. Freehold schemes and leasehold schemes',
 'Heading 5: 9. Lots — strata schemes and surveystrata schemes',
 'Heading 5: 10. Common property',
 'Heading 5: 11. Subdivision of land by strata titles scheme',
 'Heading 5: 12. Registration of strata titles scheme',
 'Heading 5: 13. Strata titles',
 'Heading 5: 14. Strata company',
 'Heading 2: Part 3 — Planning and development',
 'Heading 3: Division 1 — Planning approvals',
 'Heading 4: Subdivision 1 — Strata schemes',
 'Heading 5: 15. Subdivision approval of strata scheme',
 'Heading 5: 16. Application of Planning and Development Act',
 'Heading 4: Subdivision 2 — Surveystrata schemes',
 'He

## Chunk up the legislation

In [4]:
import re

def read_document_sections(file_path, n=5):
    '''Break document at headings up to level n (5) and return a plain text 
       document with paragraphs seperated by two newlines (\n\n)'''
    
    doc = docx.Document(file_path)
    skip_toc=True
    sections = []
    current_section = {'heading': "Document", 'level': 0, 'content': ""}

    for paragraph in doc.paragraphs:
        text = re.sub(r"\s+", ' ', paragraph.text)
        if paragraph.style.name.startswith(tuple(f"Heading {i+1}" for i in range(n))) or \
            paragraph.text.startswith(("Schedule", "Notes", "Defined terms")) or \
                re.search(r'^\d+\.', paragraph.text):
            #save old section 
            if current_section['heading'] or current_section['content']:
                sections.append(current_section)
            
            # and start a new section
            current_section = {'heading' : text,
                               #'level'   : int(re.search("Heading (\d+)", paragraph.style.name).group(1)),
                               'content' : text
                            }
        else:
            # join this paragraph text to prior ones in this section
            current_section['content'] = "\n\n".join([current_section['content'], text])

    # Add the last section
    if  current_section['heading'] or current_section['content']:
        sections.append(current_section)

    #Return list of sections
    return sections 

from langchain.schema import Document
def makeDocs(n):
    '''Break legistation by headings down to level n. This chunks up the 
       document to sizes chatGPT can digest while ensuring the clauses in
        the legislation are kept together '''

    return [Document(page_content = section['content'], metadata = {'title':section['heading']}) 
                for section in read_document_sections(legislation_path, n) ]


def counts(texts):
  '''Create some basic statistics on the corpus'''

  if len(texts) == 0:
    print("No texts")
    return

  charCounts = [len(text.page_content) for text in texts]
  wordCounts = [len(text.page_content.split()) for text in texts]
  print(f"There are {len(texts)} chunks\nAverage character count {sum(charCounts)/len(charCounts):.0f}\nAverage word count {sum(wordCounts)/len(wordCounts):.0f}")
  

In [5]:
chunk_H5 = makeDocs(5)[325:] #drop toc and title
counts(chunk_H5)

There are 380 chunks
Average character count 1343
Average word count 235


In [6]:

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
chunk_H5_split = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300).split_documents(chunk_H5)
counts(chunk_H5_split)

There are 512 chunks
Average character count 1053
Average word count 184


## Create the Pinecone database
Initialise the pinecode instance base on the API keys in .env. 

Depending on the user input, use the existing index or create a new one from the documents. Create a "similarity" document retriever based on the database.

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
import os

pinecone.init(
    api_key= os.environ.get('PINECONE_API_KEY') ,  # find at app.pinecone.io
    environment=os.environ.get('PINECONE_ENV')     # next to api key in console
)

def create_namespace(namespace, documents, embeddings):
    
    INDEX = os.environ.get('INDEX')
    
    if INDEX not in pinecone.list_indexes():
        print(f"Creating new index {INDEX}")
        pinecone.create_index(INDEX, dimension=1536)
    
    pinecone.Index(INDEX).delete(namespace=namespace, deleteAll=True)
    
    return Pinecone.from_documents(documents, 
                                   index_name=os.environ.get('INDEX'), 
                                   namespace=namespace, 
                                   embedding=embeddings)

db5 = create_namespace("SCA_H5", chunk_H5_split, 
                       embeddings= OpenAIEmbeddings())

/home/kmcisaac/Projects/Personal/SCAWA/.venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Testing retrieval

# Create and test SImon


Define a Q&A chain that 'stuffs' the retrieved chunks into the prompt to provide context. Using OpenAI deterministic (temperature=0) model `gpt-3.5-turbo`.  According to OpenAI 'gpt-3.5-turbo' is the 
> Most capable GPT-3.5 model and optimized for chat at 1/10th the cost of text-davinci-003.

In [1]:
import pinecone
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

pinecone.init(
    api_key= os.environ.get('PINECONE_API_KEY') ,  # find at app.pinecone.io
    environment=os.environ.get('PINECONE_ENV')     # next to api key in console
)

db = Pinecone.from_existing_index(index_name=os.environ.get('INDEX'), 
                                   namespace='SCA_H5', 
                                   embedding=OpenAIEmbeddings())

/home/kmcisaac/Projects/Personal/SCAWA/.venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from langchain.prompts import PromptTemplate
prompt_template = """
You are a helpful Strata legal expert in Western Australia answering questions about the "Strata Titles Act 1985" from a lot owner.

Start the answer with "An owner should always refer to their bylaws and strata plan in conjenction with the legislation".

Provide a detailed answer using the information from the legislation provided below. List relevant sections of the act. 

Do not make up answers. If you do not know say "I do not know".

{context}

Question: {question}

Answer in plain english"
"""
PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["context", "question"])

retriever=db.as_retriever(search_type="similarity", 
                            search_kwargs={"k":4})

#retriever=db.as_retriever(search_type="similarity_score_threshold", 
#                          search_kwargs={"k":3, "score_threshold":0.5})

qa = RetrievalQA.from_chain_type(
                    llm=ChatOpenAI(temperature=0), # uses 'gpt-3.5-turbo' which is cheaper and better 
                    chain_type="stuff", 
                    retriever=retriever, 
                    chain_type_kwargs={"prompt": PROMPT}, 
                    return_source_documents=True)

In [21]:
topics = db.similarity_search_with_score("I want to keep a animal", k = 4, namespace='SCA_H5')
topics

[(Document(page_content='12. Additional duties of owners and occupiers\n\n An owner or occupier of a lot must not —\n\n (a) use the lot for a purpose that may be illegal or injurious to the reputation of the building; or\n\n (b) make undue noise in or about the lot or common property; or\n\n (c) keep animals on the lot or the common property after notice in that behalf given to that person by the council.\n\n [Bylaw 12 inserted: No. 58 of 1995 s. 88(5); amended: No. 74 of 2003 s. 112(22); No. 30 of 2018 s. 110.]', metadata={'title': '12. Additional duties of owners and occupiers'}),
  0.7450912),
 (Document(page_content='8. Storage of inflammable liquids etc.\n\n An owner or occupier of a lot must not, except with the written approval of the strata company, use or store on the lot or on the common property any inflammable chemical, liquid or gas or other inflammable material, other than chemicals, liquids, gases or other materials used or intended to be used for domestic purposes, or a

In [6]:
qa("I want to make alterations.")

{'query': 'I want to make alterations.',
 'result': 'An owner should always refer to their bylaws and strata plan in conjunction with the legislation. According to the Strata Titles Act 1985, an owner must not alter the structure of their lot without giving written notice to the strata company at least 14 days before the commencement of the alteration. The owner must also obtain prior written approval from the owner of the other lot and the strata company, expressed by resolution without dissent, for a structural alteration of the lot in a strata scheme. For a surveystrata scheme, the owner must obtain prior written approval from the owner of the other lot and the strata company, expressed by resolution without dissent, and conform to plot ratio restrictions or open space requirements for the lot. The Tribunal may exempt a particular structural alteration to the lot from the application of this Division if it is reasonable and will not cause any significant inconvenience or detriment t

In [5]:
qa({"query": "I want to make alterations."})

{'query': 'I want to make alterations.',
 'result': 'An owner should always refer to their bylaws and strata plan in conjunction with the legislation. According to the Strata Titles Act 1985, an owner must not alter the structure of their lot without giving written notice to the strata company at least 14 days before the commencement of the alteration. The owner must also obtain prior written approval from the owner of the other lot and the strata company, expressed by resolution without dissent, for a structural alteration of the lot in a strata scheme other than a 2-lot scheme. For a 2-lot scheme, the owner must obtain prior written approval from the owner of the other lot and, for a leasehold scheme, the owner of the leasehold scheme. The owner may apply to the Tribunal for an exemption from the application of this Division, but the Tribunal can only make an order if it is satisfied that the structural alteration is reasonable and will not cause any significant inconvenience or detr

In [ ]:
from IPython.display import display, Markdown

import textwrap

def wrap_text_preserve_newlines(text, width=110):

    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response, sources=True, content=False):
    display(Markdown(wrap_text_preserve_newlines(llm_response['result'])))
    if sources:
      display(Markdown('\n\nSources:'))
      for source in llm_response["source_documents"]:
        display(Markdown(f"{source.metadata.get('title')} ({source.metadata.get('score')})"))
        if content:
          display(Markdown(f'{wrap_text_preserve_newlines(source.page_content)}'))

def Simon(query, sources=True, content=False):
  
  result = qa(query)
  process_llm_response(result, sources=sources, content=content)
  return (result)

In [ ]:
result = Simon("What does the legisltation cover", sources=True, content=False)

The Strata Titles Act provides rules for dividing land into strata titles schemes, which involves creating
certificates of title for individual lots within the scheme. The Act also includes provisions for the
governance and operation of strata titles schemes and for strata managers. The Act must be read together with
the Planning and Development Act 2005 and the Transfer of Land Act 1893 to fully understand the legislative
framework for this type of subdivision. The Act also allows for the creation of regulations to prescribe
matters related to the registration of plans and documents, fees, and other requirements for strata titles
schemes.



Sources:

6. Legislative framework (None)

30. Transitional provisions as to insurance (None)

224. Regulations (None)

In [ ]:
result = Simon("I am an owner in a 250 lot complex. I want to renovate. Do I need approval? How long do I need to wait", 
               sources=True, content=True)

Yes, you need approval to renovate your lot. You must submit an application for approval of the structural
alteration of your lot, which must include details of the proposal and other prescribed information. The
strata company must vote on the application within 35 days of receiving it, or you can convene a general
meeting if they do not. If the strata company does not object to the alteration within 77 days, they are taken
to have approved it. You must also give written notice of the proposed alteration to the strata company at
least 14 days before commencing the alteration.



Sources:

89. Approvals and objections to structural alterations (None)

89. Approvals and objections to structural alterations

 (1) An application for the approval of the structural alteration of a lot must set out details of the
proposal and such other information as may be prescribed.

 (2) If an application is made to a strata company under subsection (1), voting on the application must open
within 35 days after the application is received (the allowed period).

 (3) If voting on the application does not open as required by subsection (2), the applicant may convene a
general meeting, in the same manner as nearly as possible as that in which meetings are to be convened by the
council, and submit the application to that meeting.

 (4) Despite subsection (2), a council may submit an application to a general meeting convened by the council
after the allowed period if that meeting is held before a meeting is convened by the applicant under
subsection (3).

 (5) The owner of a lot or the owner of a leasehold scheme is taken to have approved the structural alteration
of a lot as set out in an application for approval served on the owner if —

 (a) the owner serves on the applicant written consent to the alteration; or

 (b) the owner has not, at the end of 42 days after being given the application, made a written objection to
the alteration; or

 (c) for a strata scheme, the owner has made such an objection but the objection does not specify the grounds
of the objection or the grounds specified are not grounds on which the owner may object under section 87.

 (6) A strata company is taken to have approved the structural alteration of a lot as set out in an
application for approval served on the strata company if —

 (a) the strata company serves on the applicant written consent to the alteration expressed by resolution
without dissent; or

 (b) despite section 87(2) —

 (i) the strata company has not, at the end of 77 days after being given the application, made a written
objection to the alteration; or

13. Notice of alteration to lot (None)

13. Notice of alteration to lot

 An owner of a lot must not alter or permit the alteration of the structure of the lot except as may be
permitted and provided for under the Act and the bylaws and in any event must not alter the structure of the
lot without giving to the strata company, not later than 14 days before commencement of the alteration, a
written notice describing the proposed alteration.

 [Bylaw 13 inserted: No. 58 of 1995 s. 88(5); amended: No. 30 of 2018 s. 111.]

35. Requirements for registration of amendment of scheme plan (None)

35. Requirements for registration of amendment of scheme plan

 (1) An amendment of a scheme plan for a strata titles scheme must not be registered unless —

 (a) for a leasehold scheme, the owner of the leasehold scheme is the applicant for registration or has given
written consent to the amendment; and

 (b) to the extent that the amendment gives effect to a type 1 subdivision —

 (i) the subdivision is authorised by resolution without dissent of the strata company; and

 (ii) each owner of a lot affected by the amendment who is not an applicant for registration of the amendment
—

 (I) has been given notice in the approved form of the subdivision and any associated amendment of the
schedule of unit entitlements; and

 (II) has given written consent to the amendment;

 and

 (iii) if the owner of a lot affected by the amendment holds a life estate in the land, the person who holds
the remainder or reversionary interest in the land —

 (I) has been given notice in the approved form of the subdivision and any associated amendment of the
schedule of unit entitlements; and

 (II) has given written consent to the amendment;

 and

 (iv) each designated interest in land that is to become common property has been discharged, surrendered,
withdrawn or otherwise extinguished;

 and

 (c) to the extent that the amendment gives effect to a type 2 subdivision —

 (i) the subdivision is authorised by resolution without dissent of the strata company; and

 (ii) the holder of each designated interest over the whole or a part of the parcel has been given notice in
the approved form of the subdivision and any associated amendment of the schedule of unit entitlements and —

 (I) has given written consent to the subdivision; or

 (II) has not, at the end of 60 days after being given notice, made a written objection to the subdivision
setting out the reasons for the objection;

 and

 (d) to the extent that the amendment gives effect to a type 3 subdivision —

In [ ]:
result = Simon("I am an owner in a 250 lot complex. I want to own a pet. Do I need approval? How long do I need to wait", 
               sources=True, content=True)